In [58]:
#导包
import pandas as pd
import numpy as np
import json

In [59]:
# #读入数据
# data = './housing.data'
# data = np.fromfile(data,sep=' ')
# data.shape #数组维度
# data[0].dtype #数据类型

In [60]:
# # 读入之后的数据被转化成1维array，其中array的第0-13项是第一条数据，第14-27项是第二条数据，以此类推.... 
# # 这里对原始数据做reshape，变成N x 14的形式
# feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE','DIS', 
#                 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
# feature_num = len(feature_names)
# # data = data.reshape([feature_num,data.shape[0] // feature_num])
# data = data.reshape([data.shape[0] // feature_num, feature_num])
# #data = np.transpose(data)
# data.shape

In [61]:
# 计算皮尔逊系数
# print(np.corrcoef(data))
# data = np.corrcoef(data)
#绘制热力图
# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.heatmap(data = data,#数据
#             cmap = 'YlGnBu',#填充色
#             #linewidths = 10,#宽度
#             #annot = True,#显示数值
#             fmt = '.1e' ,#科学计数法显示
#             xticklabels = True,
#             yticklabels = True
#             )
# plt.show()

In [62]:
#计算皮尔逊系数
# a = pd.Series([1,2,3,4,5,6,7,8,9,10])
# b = pd.Series([2,4,1,5,1,3,6,2,7,0])
# c = pd.Series([0,3,2,1,4,7,1,9,6,2])
# x = np.vstack((a,b,c))
# print(x.shape)
# r = np.corrcoef(x)
# print(r)

In [63]:
# # 数据集划分
# ratio = 0.8
# offset = int(data.shape[0] * ratio)
# training_data = data[:offset]
# training_data.shape

In [64]:
# # 计算train数据集的最大值，最小值，平均值
# maximums, minimums, avgs = \
#                     training_data.max(axis=0), \
#                     training_data.min(axis=0), \
#     training_data.sum(axis=0) / training_data.shape[0]
# # 对数据进行归一化处理
# for i in range(feature_num):
#     #print(maximums[i], minimums[i], avgs[i])
#     data[:, i] = (data[:, i] - avgs[i]) / (maximums[i] - minimums[i])
# data

In [65]:
# 数据处理封装成函数
def load_data():
    # 从文件导入数据
    datafile = './housing.data'
    data = np.fromfile(datafile, sep=' ')

    # 每条数据包括14项，其中前面13项是影响因素，第14项是相应的房屋价格中位数
    feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', \
                    'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
    feature_num = len(feature_names)

    # 将原始数据进行Reshape，变成[N, 14]这样的形状
    data = data.reshape([data.shape[0] // feature_num, feature_num])

    # 将原数据集拆分成训练集和测试集
    # 这里使用80%的数据做训练，20%的数据做测试
    # 测试集和训练集必须是没有交集的
    ratio = 0.8
    offset = int(data.shape[0] * ratio)
    training_data = data[:offset]

    # 计算训练集的最大值，最小值，平均值
    maximums, minimums, avgs = training_data.max(axis=0), training_data.min(axis=0), \
                                training_data.sum(axis=0) / training_data.shape[0]

    # 对数据进行归一化处理
    for i in range(feature_num):
        #print(maximums[i], minimums[i], avgs[i])
        data[:, i] = (data[:, i] - avgs[i]) / (maximums[i] - minimums[i])

    # 训练集和测试集的划分比例
    training_data = data[:offset]
    test_data = data[offset:]
    return training_data, test_data

In [66]:
# 获取数据
training_data,test_data = load_data()
x = training_data[:,:-1]
y = test_data[:,-1:]

In [67]:
# 查看数据
print(x[0])
print(y[0])

[-0.02146321  0.03767327 -0.28552309 -0.08663366  0.01289726  0.04634817
  0.00795597 -0.00765794 -0.25172191 -0.11881188 -0.29002528  0.0519112
 -0.17590923]
[-0.34834983]


模型设计是深度学习模型关键要素之一，也称为网络结构设计，相当于模型的假设空间，即实现模型“前向计算”（从输入到输出）的过程。

如果将输入特征和输出预测值均以向量表示，输入特征xxx有13个分量，yyy有1个分量，那么参数权重的形状（shape）是13×113\times113×1。假设我们以如下任意数字赋值参数做初始化：

w=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,−0.1,−0.2,−0.3,−0.4,0.0]

In [68]:
# # 模型设计
# w = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, -0.1, -0.2, -0.3, -0.4, 0.0]
# w = np.array(w).reshape([13, 1])

In [69]:
# # 取出第1条样本数据，观察样本的特征向量与参数向量相乘的结果。
# x1=x[0]
# t = np.dot(x1, w)
# print(t)

完整的线性回归公式，还需要初始化偏移量b，同样随意赋初值-0.2。那么，线性回归模型的完整输出是z=t+b，这个从特征和参数计算输出值的过程称为“前向计算”。

In [70]:
# b = -0.2
# z = t + b
# print(z)

In [71]:
#将计算预测输出的过程用“类和对象”的方式来描述
class Network(object):
    def __init__(self, num_of_weights):
        # 随机产生w的初始值
        # 为了保持程序每次运行结果的一致性，
        # 此处设置固定的随机数种子
        np.random.seed(0)
        self.w = np.random.randn(num_of_weights, 1)
        # print(self.w)
        self.b = 0.
        
    def forward(self, x):
        z = np.dot(x, self.w) + self.b
        return z
    
    def loss(self, z, y):
        error = z - y
        cost = error * error
        cost = np.mean(cost)
        return cost

In [72]:
# 模型计算
net = Network(13)
x1 = x[0]
y1 = y[0]
z = net.forward(x1)
print(z)
loss = net.loss(z,y)
print(loss)

[-0.63182506]
0.2164162191886279
